# From Pytorch & Co to Plug_ai

## Using conventionnal Pytorch + MonAI

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


### Imports

This cell imports necessary libraries and modules for building a deep learning pipeline with PyTorch and MONAI, including data transformations, the DynUNet model, loss function, metric, and optimizer.


In [ ]:
import os
import glob
import random
import torch
import numpy as np
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    ToTensord,
    EnsureChannelFirstd,
    ConcatItemsd,
    SpatialCropd,
    AsDiscreted
)
import monai
from monai.data import Dataset
from monai.networks.nets import DynUNet
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from torch.optim import SGD
from monai.utils import set_determinism, first

from torch.utils.data import DataLoader


### Fix random seeds
To ensure reproducibility in your experiments using PyTorch and MONAI, you'll want to set seeds for the different random number generators. This includes random seeds for Python's built-in random library, NumPy, PyTorch, and MONAI.

Here's an example of how to set seeds for these libraries:

In [ ]:
def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    monai.utils.set_determinism(seed)

# Set the seed value
seed = 42
set_all_seeds(seed)

Now you've set the seeds for all the relevant libraries. Remember, even though setting seeds helps improve reproducibility, there still might be some non-deterministic behavior due to GPU operations or other factors outside your control.

### Dataset

This cell defines the data directory, a function to read and process the data list, sets up a data transformation pipeline using MONAI's Compose, and creates a training dataset with the first 20 samples from the data list. The transformation pipeline includes loading images, ensuring channels are in the correct order, concatenating input channels, cropping the input and label, and converting the label to one-hot encoding.

In [ ]:
data_dir = "/gpfswork/rech/ibu/commun/BraTS2021/BraTS2021_Training_Data/"

def get_datalist(dataset_dir):
    datalist = []
    with open(os.path.join(dataset_dir, "train.txt"), "r") as f:
        lines = f.readlines()
        for line in lines:
            file_dic = {}
            files = line.split()
            for i, file in enumerate(files[:-1]):
                file_dic[f"channel_{i}"] = os.path.join(dataset_dir, file)

            file_dic["label"] = os.path.join(dataset_dir, files[-1])
            datalist.append(file_dic)
    return datalist

datalist = get_datalist(data_dir)
keys = list(datalist[0].keys())

transform = Compose([
            LoadImaged(keys=keys),
            EnsureChannelFirstd(keys=keys),
            ConcatItemsd(keys[:-1], "input"),
            SpatialCropd(keys=['input', 'label'], # crop it to make easily usable for etape 1
                         roi_size=[128, 128, 128],
                         roi_center=[0, 0, 0]
                         ),
            AsDiscreted(keys=['label'], to_onehot=5)
        ])



train_dataset = Dataset(data=datalist[:20],
                        transform=transform)

### Dataloader
This cell creates a DataLoader for the training dataset, with a batch size of 2, enabling shuffling for random sampling, using 4 worker processes for parallel data loading, and setting a prefetch factor of 10 for efficient data loading in the pipeline.

In [ ]:
train_loader = DataLoader(train_dataset, 
                          batch_size=2, 
                          shuffle=True, 
                          num_workers=4,
                         prefetch_factor=10)

### Model
This cell initializes the DynUNet model with the specified parameters and moves it to the available device (either GPU or CPU). The model is configured for 3D input, 4 input channels, 5 output channels, custom kernel sizes, strides, upsample kernel sizes, instance normalization, and deep supervision with 3 supervision layers.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = DynUNet(spatial_dims = 3,
                in_channels = 4,
                out_channels = 5,
                kernel_size = [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]],
                strides = [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]],
                upsample_kernel_size = [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]],
                norm_name = "instance",  # you can use fused kernel for normal layer when set to `INSTANCE_NVFUSER`
                deep_supervision =  True,
                deep_supr_num = 3,).to(device)


### Criterion
This cell defines a combined Dice loss and cross-entropy loss function using MONAI's DiceCELoss, with one-hot encoding for target labels and softmax activation for the model's output.

In [ ]:
loss_function = DiceCELoss(to_onehot_y=True, 
                           softmax=True)

### Metric
This cell defines the evaluation metric as the Dice metric using MONAI's DiceMetric. It excludes the background class and computes the mean Dice score across all classes.


In [ ]:
metric = DiceMetric(include_background=False, 
                    reduction="mean")

### Optimizer
This cell initializes the Stochastic Gradient Descent (SGD) optimizer with a learning rate of 0.001 for the model's parameters.


In [ ]:
optimizer = SGD(model.parameters(), lr=0.001)

### Training Loop

This cell trains the model for 2 epochs using the DataLoader, model, loss function, and optimizer previously defined. It iterates through the training data, computes the loss, and updates the model parameters. The training loss is accumulated and the average loss for each epoch is printed.

In [ ]:
num_epochs = 2
val_interval = 2

best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []

for epoch in range(num_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{num_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = (
            batch_data["input"].to(device),
            batch_data["label"].to(device),
        )
        
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.unbind(outputs, dim=1)[0]
        
        labels = torch.argmax(labels, dim=1, keepdim=True)
        loss = loss_function(outputs, labels)
    
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"{step}/{len(train_dataset) // train_loader.batch_size}, train_loss: {loss.item():.4f}")
    
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

## Plug_AI API

### Config file

In [46]:
%%writefile config_demo.yaml
# Default config is used even if no config file is specified in CLI. Undefined arguments will take the default values.
##################################################################################################
####################################### Global arguments : #######################################
##################################################################################################
config_file: null
export_config: null
mode: TRAINING # Choose between Training, Evaluation, Inference
verbose: FULL #Full, Restricted, None
seed: null

Overwriting config_demo.yaml


### Dataset & Dataloader

In [47]:
%%writefile -a config_demo.yaml
##################################################################################################
######################################## Data arguments : ########################################
##################################################################################################
dataset: MedNIST
dataset_kwargs:
    dataset_dir: /gpfswork/rech/ibu/commun/datasets/MedNIST # Absolute path to the dataset root dir
    download_dataset: false
    transformation: MedNIST_transform
preprocess: null
preprocess_kwargs:
train_ratio: 1 #How to specify the split? Train ratio => Dataset => train+val (train_ratio) | test (1 - train_ratio)
val_ratio: 0.2 #A subfraction of the train set to use for validation (train_ratio * val_ratio = val_real_ratio)
limit_sample: 20
batch_size: 2
shuffle: true
drop_last: true

Appending to config_demo.yaml


### Model

In [48]:
%%writefile -a config_demo.yaml
##################################################################################################
####################################### Model arguments : ########################################
##################################################################################################
model: DenseNet     #model_type MODEL_TYPE
model_kwargs:     #model_args MODEL_ARGS
    spatial_dims: 2
    in_channels: 1
    out_channels: 6
    img_size: 64    

Appending to config_demo.yaml


### Training configuration

In [49]:
%%writefile -a config_demo.yaml
##################################################################################################
##################################### Execution arguments : ######################################
##################################################################################################
#Training settings
nb_epoch: 2
device: cuda
random_seed: 2022  # None for real randomness, set an integer for reproductibility
report_log: False

loop: Default
optimizer: SGD
optimizer_kwargs:
    lr: 0.0001
    momentum: 0.99
    weight_decay: 3e-5
    nesterov: True
lr_scheduler: None
lr_scheduler_kwargs:
    step_size: 2
    verbose: True

criterion: DiceCELoss
criterion_kwargs:
    to_onehot_y: False
    softmax: True

metric: None
metric_kwargs:

Appending to config_demo.yaml


### Execution

In [2]:
!python -m plug_ai --config_file config_demo.yaml

/gpfslocalsup/pub/anaconda-py3/2021.05/envs/pytorch-gpu-1.10.1+py3.9.7/bin/python: No module named plug_ai


In [3]:
!pip list | grep plug_ai